# Odporúčanie

In [1]:
%load_ext autoreload
%autoreload all

In [2]:
# dependencies
import pandas as pd
import wandb
import glob
import random

In [3]:
#local modules
import sys
sys.path.append("..")

from _helpers import constants
from _helpers.verify_submission.verify_subm import main as verify_subm
from _helpers.score_submission.score_subm import main as score_subm

from models.model_random import ModelRandom
from models.model_nochange import ModelNoChange
from models.model_popular import ModelPopular
from models.model_log_reg import ModelLogisticRegression
from models.model_cheapest import ModelCheapest
from models.model_xgboost import ModelXGBoost
from models.model_mlp import ModelMLP
from models.model_neural import ModelNeural

2023-05-10 19:58:22.154431: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 19:58:22.258697: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-10 19:58:22.262415: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-05-10 19:58:22.262444: I tensorf

Iitialize wandb - logging tool

In [4]:
wandb.login()

wandb: Currently logged in as: mcfreddie777. Use `wandb login --relogin` to force relogin


True

Load the data

In [5]:
df_train = pd.read_parquet(constants.DROPPED_TRAIN)
df_test = pd.read_parquet(constants.DROPPED_TEST)

In [6]:
models = {
    'nochange': ModelNoChange(),
    'random': ModelRandom(),
    'popular': ModelPopular(),
    'cheapest': ModelCheapest(),
    'log-reg': ModelLogisticRegression(),
    'xgboost': ModelXGBoost(),
    'mlp': ModelMLP(),
    'neural': ModelNeural(),
}

Choose the correct model and setup parameters of the model

In [26]:
# Tinker with the parameters
run = 3
notes = 'With randomizedsearch'
params = {
    'model': 'xgboost',
    'features': [
        "impressed_item_position",
        "impressed_item_rating",
        "user_impressed_item_interaction_count",
        "price",
        "price_above_impression_mean",
        "is_last_interacted",
    ]
}

wandb_run = wandb.init(entity='mcfreddie777', project="dp-recsys", name=f'model_{params["model"]}_run_{run}',notes=notes)
wandb_run.config.update(params)

model = models[params['model']]
model.update(params)
model.fit(df_train)

Predikujeme odporúčania

In [27]:
df_recommendations = model.predict(df_test)

Verifikujeme predikcie

In [28]:
verify_subm(df_subm=df_recommendations, df_test=df_test)

Checking for required columns in the submission file...
> check passed
Checking for duplicate sessions in the submission file...
> check passed
Checking that all the required sessions are present in submission...
> check passed
All checks passed


In [29]:
df_recommendations.to_csv(constants.OUTPUT_DIR / f'submission_popular_{params["model"]}_{run}.csv', index=False)

Vypočítame si MRR na dátach

In [30]:
df_gt = pd.read_csv(constants.GROUND_TRUTH)
mrr, map3 = score_subm(df_subm=df_recommendations, df_gt=df_gt)
wandb_run.log({"mrr": mrr, "map3": map3})

Mean reciprocal rank:      0.8329
Mean average precision @3: 0.311


In [31]:
wandb_run.finish()

map3,▁
mrr,▁
map3,0.311
mrr,0.8329
